# Shipment File

In [ ]:
import datetime as dt
import pandas as pd
import time
df = pd.read_excel(
    r'C:\Users\KohMansf\Desktop\MJ Projects\All Files\Waterfall\SHIPMENT\Shipment_File.xlsx',
    sheet_name='Sheet2')

# Cleaning Data

**Filling blanks (ISO_WK_NR) with actual week number. In this case [WkDate 43374] = [ISO_WK_NR 40]. Fill in XX at code .fillna(XX) with actual week number**

In [ ]:
df['ISO_WK_NR'] = df['ISO_WK_NR'].fillna(40)
df['ISO_WK_NR'] = df['ISO_WK_NR'].astype(int)

**Subtracting 1 from ISO_WK_NR then replacing week 0 with week 52**

In [ ]:
df['ISO_WK_NR'] = df['ISO_WK_NR'] - 1
df['ISO_WK_NR'] = df['ISO_WK_NR'].replace(0, 52)

**Creating Year**

In [ ]:
df['Year'] = pd.TimedeltaIndex(df['WkDate'], unit='d') + dt.datetime(
    1899, 12, 30)

df['Year'] = df['Year'].dt.year

**Creating Planning_Week**

In [ ]:
df[['Year', 'ISO_WK_NR']] = df[['Year', 'ISO_WK_NR']].astype('str')
df['Planning_Wk'] = df[['Year', 'ISO_WK_NR']].apply(lambda x: 'W'.join(x),
                                                    axis=1)

In [ ]:
df = df.rename(columns={'Total':'Qty'})
df.head()

**Map Region base on Target Location (Using Original File) , fill blanks with Region column if cannot map on Original File**

In [ ]:
region_dict = {
    'D735': 'NA',
    'D795': 'NA',
    'DV02LM': 'NA',
    'DHNLLM': 'EMEA',
    'N8LC': 'EMEA',
    '01CR': 'EMEA',
    'N8DB': 'EMEA',
    '01CT': 'EMEA',
    '01DQ': 'EMEA',
    '01BM': 'EMEA',
    'B301': 'LA',
    'H501': 'LA',
    'DV02': 'LA',
    'H503': 'LA',
    'H5V1': 'LA',
    'N101': 'LA',
    'Z402': 'LA',
    'H415': 'APJ',
    'H416': 'APJ',
    'WW': 'Region'
}

In [ ]:
df['Region_Ori'] = df['Target Location'].map(region_dict)

df['Region_Ori'] = df['Region_Ori'].fillna(df['Region'])

df.head()

**Creating Primary Key**

In [ ]:
# Change to FK From PK
df[['Region_Ori', 'SKU', 'WkDate',
    'Planning_Wk']] = df[['Region_Ori', 'SKU', 'WkDate',
                          'Planning_Wk']].astype('str')

df['FK'] = df['Region_Ori'] + df['SKU'] + df['WkDate'] + df['Planning_Wk']

**Drop irrelevant columns**

In [ ]:
df = df.drop(columns=['Year', 'Region'])

df.head()

# POR File

**Map SKU of Shipment File to MPA**

In [ ]:
por_df = pd.read_csv(
    r'C:\Users\KohMansf\Desktop\MJ Projects\All Files\Waterfall\2019POR\2019_to_upload.csv',
    sep=',',
    dtype={
        'PK': str,
        'Planning_Wk': str,
        'MPA': str,
        'SKU': str,
        'Program': str,
        'Platform': str,
        'Product_Line': str,
        'Target_Location': str,
        'WkDate': str,
        'YYYYWW': str,
        'Region': str,
        'Qty': int,
        'FK': str,
        'QtyType': str
    },
    na_filter=False)
por_df

In [ ]:
dfskumpa = por_df.groupby(['SKU', 'MPA'])['Qty'].sum().reset_index()
dfskumpa.head()

In [ ]:
mydict = pd.Series(dfskumpa.MPA.values, index=dfskumpa.SKU).to_dict()

mydict

In [ ]:
df['MPA'] = df['SKU'].map(mydict)

In [ ]:
df = df.rename(columns={'Region_Ori':'Region'})
df.head()

# Arrange Columns

In [ ]:
dffinalarr = df[[
    'FK', 'SKU', 'Platform', 'Region', 'MPA', 'Target Location', 'WkDate',
    'ISO_WK_NR', 'Qty'
]]
dffinalarr.head()

In [ ]:
new = df['Planning_Wk'].str.split('W', n=1, expand=True)
dffinalarr['YYYY'] = new[0]
dffinalarr['WW'] = new[1]
dffinalarr.head()

In [ ]:
dffinalarr["WW"] = dffinalarr["WW"].astype(int)
dffinalarr["WW"] = dffinalarr["WW"] + 1
dffinalarr["WW"] = dffinalarr['WW'].map("{:02}".format)
dffinalarr["WW"] = dffinalarr["WW"].astype(str)
dffinalarr['YYYYWW'] = dffinalarr['YYYY'] + dffinalarr['WW']
dffinalarr = dffinalarr.drop(columns=['YYYY', 'WW'])
dffinalarr['QtyType'] = 'SHIP'
dffinalarr.head()

# Final Output

In [ ]:
dffinalarr.to_csv('Shipment.csv', index=False)